# TCC - Mineração de Processos no Transporte Público de Curitiba

## Etapa 1: Aquisição e Estruturação de Dados da API URBS

O primeiro passo fundamental deste projeto é a coleta de dados brutos do sistema de transporte. Com base na documentação oficial fornecida, vamos interagir com o web service da URBS para obter a posição dos veículos.

- **Conexão com a API:** Utilizamos a biblioteca `requests` para fazer uma chamada ao endpoint `getVeiculos.php`.
- **Carregamento e Estruturação:** A resposta JSON da API é carregada em um DataFrame do pandas. Uma transposição (`.T`) é aplicada imediatamente para garantir que os veículos sejam as linhas e os atributos (`COD`, `LAT`, etc.) sejam as colunas, que é o formato padrão para análise.

#### Importação das bibliotecas

In [1]:
import requests
import pandas as pd

#### Configurações da Requisição

In [2]:
base_url = "https://transporteservico.urbs.curitiba.pr.gov.br/"
endpoint = "getVeiculos.php"
api_code = "65d35"
full_url = f"{base_url}{endpoint}"
params = {'c': api_code}

print(f"Acessando a API em: {full_url}")

Acessando a API em: https://transporteservico.urbs.curitiba.pr.gov.br/getVeiculos.php


In [3]:
try:
    # Realiza a requisição GET
    response = requests.get(full_url, params=params, timeout=30)
    response.raise_for_status()   
    data = response.json()
    print("Conexão e recebimento de dados bem-sucedidos!")
    
    # Carrega os dados e imediatamente transpõe (.T) o DataFrame para corrigir a estrutura
    df_raw = pd.DataFrame(data).T
    
    print("\n--- Estrutura do DataFrame corrigida ---")
    print(f"Formato (linhas, colunas): {df_raw.shape}")
    print(df_raw.head())

except requests.exceptions.RequestException as e:
    print(f"ERRO de Conexão: {e}")
    df_raw = None
except Exception as e:
    print(f"ERRO ao processar os dados: {e}")
    df_raw = None

Conexão e recebimento de dados bem-sucedidos!

--- Estrutura do DataFrame corrigida ---
Formato (linhas, colunas): (1542, 12)
         COD REFRESH         LAT         LON CODIGOLINHA ADAPT TIPO_VEIC  \
HI607  HI607   18:54  -25.485753  -49.346446         826     0         4   
HC013  HC013   18:54   -25.51415  -49.336483         826     1         1   
JC301  JC301   18:54  -25.453428  -49.361171         826     1         3   
JI299  JI299   18:54  -25.441586  -49.346648         826     1         1   
HI006  HI006   18:54  -25.505995  -49.340305         826     1         1   

      TABELA    SITUACAO        SITUACAO2   SENT TCOUNT  
HI607    4-2    ATRASADO  REALIZANDO ROTA    IDA      1  
HC013    1-2    ATRASADO  REALIZANDO ROTA    IDA      1  
JC301      3  NO HORÁRIO  REALIZANDO ROTA    IDA      1  
JI299    6-2   ADIANTADO  REALIZANDO ROTA    IDA      1  
HI006    2-2  NO HORÁRIO  REALIZANDO ROTA  VOLTA      1  


## Etapa 2: Preparação e Limpeza dos Dados

Com o DataFrame `df_raw` corretamente estruturado, esta etapa foca em prepará-lo para a análise.

- **Renomeação de Colunas:** Alteramos os nomes originais da API para um padrão mais claro e consistente (ex: `COD` para `vehicle_id`).
- **Criação do Timestamp:** Criamos uma coluna `timestamp` completa, combinando a data atual com o horário da API, e a convertemos para o formato `datetime` do pandas.

In [4]:
# Este código assume que a célula da Etapa 1 foi executada e a variável 'df_raw' existe.

if 'df_raw' in locals() and df_raw is not None:
    # 1. Renomeando as colunas
    column_mapping = {
        'COD': 'vehicle_id',
        'REFRESH': 'refresh_time',
        'LAT': 'latitude',
        'LON': 'longitude',
        'CODIGOLINHA': 'line_id',
        'ADAPT': 'adapted_vehicle',
        'TIPO_VEIC': 'vehicle_type',
        'TABELA': 'schedule_table',
        'SITUACAO': 'situation',
        'SITUACAO2': 'situation_2',
        'SENT': 'direction',
        'TCOUNT': 't_count'
    }
    df_cleaned = df_raw.rename(columns=column_mapping)
    print("--- Colunas após renomear ---")
    print(df_cleaned.columns.tolist())
    print("\n")

    # 2. Criando a coluna de Timestamp
    today_date_str = str(pd.to_datetime('today').date())
    df_cleaned['timestamp'] = pd.to_datetime(today_date_str + ' ' + df_cleaned['refresh_time'], 
                                             format='%Y-%m-%d %H:%M')

    print("--- Verificação dos tipos de dados (note a coluna 'timestamp') ---")
    df_cleaned.info()
    print("\n")
    
    print("--- Visualização do DataFrame limpo e pronto ---")
    print(df_cleaned.head())
else:
    print("ERRO: A variável 'df_raw' não foi encontrada. Execute a célula da Etapa 1 primeiro.")

--- Colunas após renomear ---
['vehicle_id', 'refresh_time', 'latitude', 'longitude', 'line_id', 'adapted_vehicle', 'vehicle_type', 'schedule_table', 'situation', 'situation_2', 'direction', 't_count']


--- Verificação dos tipos de dados (note a coluna 'timestamp') ---
<class 'pandas.core.frame.DataFrame'>
Index: 1542 entries, HI607 to BI857
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   vehicle_id       1542 non-null   object        
 1   refresh_time     1542 non-null   object        
 2   latitude         1542 non-null   object        
 3   longitude        1542 non-null   object        
 4   line_id          1542 non-null   object        
 5   adapted_vehicle  1542 non-null   object        
 6   vehicle_type     1542 non-null   object        
 7   schedule_table   1542 non-null   object        
 8   situation        1542 non-null   object        
 9   situation_2      1542 non-null   o

#### Visualização do DataFrame Transposto, Renomeado e com timestamp

In [5]:
df_cleaned

,vehicle_id,refresh_time,latitude,longitude,line_id,adapted_vehicle,vehicle_type,schedule_table,situation,situation_2,direction,t_count,timestamp
HI607,HI607,18:54,-25.485753,-49.346446,826,0,4,4-2,ATRASADO,REALIZANDO ROTA,IDA,1,2025-10-15 18:54:00
HC013,HC013,18:54,-25.51415,-49.336483,826,1,1,1-2,ATRASADO,REALIZANDO ROTA,IDA,1,2025-10-15 18:54:00
JC301,JC301,18:54,-25.453428,-49.361171,826,1,3,3,NO HORÁRIO,REALIZANDO ROTA,IDA,1,2025-10-15 18:54:00
JI299,JI299,18:54,-25.441586,-49.346648,826,1,1,6-2,ADIANTADO,REALIZANDO ROTA,IDA,1,2025-10-15 18:54:00
HI006,HI006,18:54,-25.505995,-49.340305,826,1,1,2-2,NO HORÁRIO,REALIZANDO ROTA,VOLTA,1,2025-10-15 18:54:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
HA004,HA004,18:47,-25.575016,-49.329443,REC,1,1,,,,,1,2025-10-15 18:47:00
BI895,BI895,18:54,-25.441485,-49.346135,X43,1,7,1-2,ADIANTADO,REALIZANDO ROTA,CIRCULAR,1,2025-10-15 18:54:00
HI043,HI043,18:41,-25.585808,-49.317891,690,1,1,1-2,NO HORÁRIO,REALIZANDO ROTA,IDA,3,2025-10-15 18:41:00
PI309,PI309,18:53,-25.459901,-49.196601,334,1,3,1,NO HORÁRIO,REALIZANDO ROTA,CIRCULAR,1,2025-10-15 18:53:00


## Etapa 3: Definição e Criação do Case ID

O *Case ID* é o identificador que agrupa todos os eventos pertencentes a uma única instância do processo.  Em nosso contexto, o processo é a viagem de um ônibus. Portanto, cada viagem distinta precisa de um identificador único.

Para garantir essa unicidade, vamos criar o *Case ID* concatenando três colunas:
1.  `line_id`: O código da linha do ônibus.
2.  `vehicle_id`: O identificador único do veículo.
3.  `date`: A data em que a viagem ocorreu.

Primeiro, extrairemos a data da nossa coluna `timestamp`. Em seguida, combinaremos essas três informações em uma nova coluna chamada `case_id`. Este será o pilar para agrupar e reconstruir as jornadas individuais nas etapas seguintes.

In [6]:
# Este código assume que a célula da Etapa 2 foi executada e o DataFrame 'df_cleaned' existe.

if 'df_cleaned' in locals() and df_cleaned is not None:
    # 1. Extrair a data da coluna 'timestamp' e criar uma nova coluna 'date'
    # .dt é o acessor para propriedades de datetime em uma Series do pandas
    df_cleaned['date'] = df_cleaned['timestamp'].dt.date
    
    print("--- Coluna 'date' criada ---")
    print(df_cleaned[['timestamp', 'date']].head())
    print("\n")

    # 2. Criar a coluna 'case_id' concatenando as informações
    # Convertemos todas as colunas para string antes de concatenar para evitar erros
    df_cleaned['case_id'] = (df_cleaned['line_id'].astype(str) + '-' +
                             df_cleaned['vehicle_id'].astype(str) + '-' +
                             df_cleaned['date'].astype(str))

    print("--- Coluna 'case_id' criada com sucesso ---")
    
    # Reorganizando as colunas para melhor visualização (opcional, mas recomendado)
    # Colocando os identificadores mais importantes no início.
    cols_to_move = ['case_id', 'vehicle_id', 'line_id', 'timestamp']
    df_final_structure = df_cleaned[cols_to_move + [col for col in df_cleaned.columns if col not in cols_to_move]]

    print("\n--- Visualização do DataFrame com o Case ID ---")
    print(df_final_structure.head())

else:
    print("ERRO: O DataFrame 'df_cleaned' não foi encontrado. Execute as etapas anteriores primeiro.")

--- Coluna 'date' criada ---
                timestamp        date
HI607 2025-10-15 18:54:00  2025-10-15
HC013 2025-10-15 18:54:00  2025-10-15
JC301 2025-10-15 18:54:00  2025-10-15
JI299 2025-10-15 18:54:00  2025-10-15
HI006 2025-10-15 18:54:00  2025-10-15


--- Coluna 'case_id' criada com sucesso ---

--- Visualização do DataFrame com o Case ID ---
                    case_id vehicle_id line_id           timestamp  \
HI607  826-HI607-2025-10-15      HI607     826 2025-10-15 18:54:00   
HC013  826-HC013-2025-10-15      HC013     826 2025-10-15 18:54:00   
JC301  826-JC301-2025-10-15      JC301     826 2025-10-15 18:54:00   
JI299  826-JI299-2025-10-15      JI299     826 2025-10-15 18:54:00   
HI006  826-HI006-2025-10-15      HI006     826 2025-10-15 18:54:00   

      refresh_time    latitude   longitude adapted_vehicle vehicle_type  \
HI607        18:54  -25.485753  -49.346446               0            4   
HC013        18:54   -25.51415  -49.336483               1            1   
JC

#### Visualização do DataFrame com case id

In [7]:
df_final_structure

,case_id,vehicle_id,line_id,timestamp,refresh_time,latitude,longitude,adapted_vehicle,vehicle_type,schedule_table,situation,situation_2,direction,t_count,date
HI607,826-HI607-2025-10-15,HI607,826,2025-10-15 18:54:00,18:54,-25.485753,-49.346446,0,4,4-2,ATRASADO,REALIZANDO ROTA,IDA,1,2025-10-15
HC013,826-HC013-2025-10-15,HC013,826,2025-10-15 18:54:00,18:54,-25.51415,-49.336483,1,1,1-2,ATRASADO,REALIZANDO ROTA,IDA,1,2025-10-15
JC301,826-JC301-2025-10-15,JC301,826,2025-10-15 18:54:00,18:54,-25.453428,-49.361171,1,3,3,NO HORÁRIO,REALIZANDO ROTA,IDA,1,2025-10-15
JI299,826-JI299-2025-10-15,JI299,826,2025-10-15 18:54:00,18:54,-25.441586,-49.346648,1,1,6-2,ADIANTADO,REALIZANDO ROTA,IDA,1,2025-10-15
HI006,826-HI006-2025-10-15,HI006,826,2025-10-15 18:54:00,18:54,-25.505995,-49.340305,1,1,2-2,NO HORÁRIO,REALIZANDO ROTA,VOLTA,1,2025-10-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HA004,REC-HA004-2025-10-15,HA004,REC,2025-10-15 18:47:00,18:47,-25.575016,-49.329443,1,1,,,,,1,2025-10-15
BI895,X43-BI895-2025-10-15,BI895,X43,2025-10-15 18:54:00,18:54,-25.441485,-49.346135,1,7,1-2,ADIANTADO,REALIZANDO ROTA,CIRCULAR,1,2025-10-15
HI043,690-HI043-2025-10-15,HI043,690,2025-10-15 18:41:00,18:41,-25.585808,-49.317891,1,1,1-2,NO HORÁRIO,REALIZANDO ROTA,IDA,3,2025-10-15
PI309,334-PI309-2025-10-15,PI309,334,2025-10-15 18:53:00,18:53,-25.459901,-49.196601,1,3,1,NO HORÁRIO,REALIZANDO ROTA,CIRCULAR,1,2025-10-15


## Etapa 4: Aquisição dos Dados de Pontos de Ônibus (Geofences)

Para identificar as atividades de "chegada" e "partida", precisamos primeiro de um mapa de onde essas atividades podem ocorrer. [cite_start]No nosso caso, esses locais são os pontos de ônibus.  Utilizaremos a API da URBS para obter os dados geográficos de todos os pontos de parada.

O processo será feito em duas etapas:
1.  **Obter a Lista de Todas as Linhas:** Primeiro, faremos uma requisição ao endpoint `getLinhas.php` para buscar um catálogo de todas as linhas de ônibus disponíveis.
2.  **Iterar e Coletar Pontos por Linha:** Em seguida, vamos percorrer a lista de linhas e, para cada uma, fazer uma nova requisição ao endpoint `getPontosLinha.php`. Este endpoint nos retornará as coordenadas (latitude e longitude), nome e outras informações de cada ponto de parada daquela linha.

Ao final, todos os dados dos pontos de todas as linhas serão consolidados em um único DataFrame, que chamaremos de `stops_df`. Este DataFrame será a nossa referência geoespacial para a próxima etapa de junção de dados.